# Jpeg algorithm

**Groupe 30**: Tony Heng / Gabriel Rayzal / Nathan Cabasso / Ferdinand Mom

In [1]:
# Explained what is JPEG etc ...

# Table of Contents

- Theory
    - Compression
    - Decompression
- Practical
    - Compression / Decompression
    - Different quality of compression

In [8]:
import numpy as np

from src.compression import *
from src.decompression import *
from src.utils import *

# I/ Theory

# <ins> Compression </ins>

Whole compression pipeline

![](https://cdn.discordapp.com/attachments/844292513665515531/859104978538856448/unknown.png)

## 1) Block splitting

First step is to divide input image into non-overlapping 8 × 8 macro-blocks.
![](https://cdn.discordapp.com/attachments/844292513665515531/859064441525370900/unknown.png)
If the dimensions are not divisible in integer numbers of blocks, the image can be padded:

- zero padding: #TODO
- replicating border: #TODO

A naive approach could be to pad only a side of the image (bottom-left). The downside of this approach is that we can find ourselves with macro-blocks full of zeros. Thus, we decided to implement our padding in a such way that the image is always center in the middle.

## 2) DCT

The second step is to perform a DCT (Discret Cosine Transformation) on each macro-block. To understand the need of DCT, we need to first understand what is an image. An image is the superposition of:

- Low frequencies: overall structure of image.
- High frequencies: Details (edges and noises)

An image is originally represented in its spatial representation which means the image is decomposed in a set of basis image.

![](https://cdn.discordapp.com/attachments/844292513665515531/859090327977721866/unknown.png)

The problem is then the following: How do we discard the useless part of high frequency (noises) without removing important information ?

To do so, we perform a frequency analysis of the image. However, the way the image is initially represented (in spatial representation) is not suited to carry out a frequency analysis. Indeed, high frequencies are disseminated everywhere in the image so it is hard to tell whether a quick variation in terms of pixel is due to an edge (important) or noise (to be discarded).

That is why we need to change the representation of our image which is done by performing transformations on our image in order to represent it into a new basis. More precisely, the transformation will enable to represent our image into a new basis. Here are some transformations:

- Walsh-Hadamard
- Discret Fourier Transform (DFT)
- Discret Cosine Transform (DCT)

JPEG uses DCT which succeed in compacting most of the image energy into as few coefficients as possible compare to Waslh-Hadamard and does not require to store complex number (real and imaginary parts) like DFT which requires double the memory for storage.

Each 8x8 macro-block will then be multiply with the following DCT kernel matrix:

![](https://cdn.discordapp.com/attachments/844292513665515531/859082211910221824/unknown.png)

The result of the DCT is also an 8 by 8 block where the first coefficient at the top left corner represents the DC coefficient. Remaining coefficients are called AC coefficients. The advantage of the DCT is its tendency to aggregate most of the signal in top left corner of the result. Note that the higher the coefficient, the more information it contains, useful to properly reconstruct the image. Thus, we can gain in storage by removing smaller coefficients which is the goal of next step.

In [15]:
def test_dct():
    # Taken from wikipedia: https://en.wikipedia.org/wiki/JPEG#Encoding
    block = np.array([
        [52, 55, 61, 66, 70, 61, 64, 73],
        [63, 59, 55, 90, 109, 85, 69, 72],
        [62, 59, 68, 113, 144, 104, 66, 73],
        [63, 58, 71, 122, 154, 106, 70, 69],
        [67, 61, 68, 104, 126, 88, 68, 70],
        [79, 65, 60, 70, 77, 68, 58, 75],
        [85, 71, 64, 59, 55, 61, 65, 83],
        [87,  79, 69, 68, 65, 76, 78, 94]]
    )

    expected = np.array([
        [-415.38, -30.19, -61.20, 27.24, 56.12, -20.10, -2.39, 0.46],
        [4.47, -21.86, -60.76, 10.25, 13.15, -7.09, -8.54, 4.88],
        [-46.83, 7.37, 77.13, -24.56, -28.91, 9.93, 5.42, -5.65],
        [-48.53, 12.07, 34.10, -14.76, -10.24, 6.30, 1.83, 1.95],
        [12.12, -6.55, -13.20, -3.95, -1.87, 1.75, -2.79, 3.14],
        [-7.73, 2.91, 2.38, -5.94, -2.38, 0.94, 4.30, 1.85],
        [-1.03, 0.18, 0.42, -2.42, -0.88, -3.02, 4.12, -0.66],
        [-0.17, 0.14, -1.07, -4.19, -1.17, -0.10, 0.50, 1.68]]
    )

    result = dct(block)
    assert np.allclose(result, expected, atol=1e-2, rtol=0.) == True
    print("[test_dct]: OK")

test_dct()

[test_dct]: OK


## 3) Quantization

The third step is quantization which looks for a way to maximize the number of 0 coefficients. This is done through the use of quantization matrix which were found empirically.

First quantization matrix is for RGB image while second one is for YUV image (only to be applied to UV channels): 

![](https://cdn.discordapp.com/attachments/844292513665515531/859100308390805534/quantification_Qq.png)
![](https://cdn.discordapp.com/attachments/844292513665515531/859100870369214534/quantification_Quv.png)

The formula is the following:

$$round(\frac{DCT(block - 128)}{Q})$$

- Before applying DCT, we must remove 128 to each 8x8 macro-block because statiscally, it was found that the mean value of an 8x8 macro-block is close to 128. Thus, $DCT(block - 128)$ wille result in more value close to 0.
- After dividing by the quantization matrix, we round to the nearest integer. This is the reason why JPEG is a lossy algorithm. It will not be possible to recover the same value during decompression.

One can also change the quality of compression using a quality factor $q \in [1, 100]$ and the following algorithm:

![](https://cdn.discordapp.com/attachments/844292513665515531/859104454381142046/unknown.png)

The higher $q$ is, the more similar the compressed image will be compared to the original image.

In [19]:
def test_quantization():
    # Taken from wikipedia: https://en.wikipedia.org/wiki/JPEG#Encoding
    dct_block = np.array([
        [-415.38, -30.19, -61.20, 27.24, 56.12, -20.10, -2.39, 0.46],
        [4.47, -21.86, -60.76, 10.25, 13.15, -7.09, -8.54, 4.88],
        [-46.83, 7.37, 77.13, -24.56, -28.91, 9.93, 5.42, -5.65],
        [-48.53, 12.07, 34.10, -14.76, -10.24, 6.30, 1.83, 1.95],
        [12.12, -6.55, -13.20, -3.95, -1.87, 1.75, -2.79, 3.14],
        [-7.73, 2.91, 2.38, -5.94, -2.38, 0.94, 4.30, 1.85],
        [-1.03, 0.18, 0.42, -2.42, -0.88, -3.02, 4.12, -0.66],
        [-0.17, 0.14, -1.07, -4.19, -1.17, -0.10, 0.50, 1.68]]
    )

    Q_MAT = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72,  92, 95, 98, 112, 100, 103, 99]]
    )

    expected = np.array([
        [-26, -3, -6, 2, 2, -1, 0, 0],
        [0, -2, -4, 1, 1, 0, 0, 0],
        [-3, 1, 5, -1, -1, 0, 0, 0],
        [-3, 1, 2, -1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]
    )

    result = quantization(dct_block, Q_MAT)
    assert np.allclose(result, expected) == True
    assert result.dtype == expected.dtype
    print("[test_quantization]: OK")

test_quantization()

[test_quantization]: OK


## 4) Zigzag + Huffman

### a) Zigzag

The goal of this step is to represent our 8x8 macro-block with only the essential coefficients. Indeed, after quantization, a lot of coefficients on the bottom corner of our macro-block are set to 0 while most of the important coefficients are on the top left corner.

![](https://cdn.discordapp.com/attachments/844292513665515531/859105773091291146/unknown.png)

In [20]:
def test_zigzag():
    # Taken from wikipedia: https://en.wikipedia.org/wiki/JPEG#Encoding
    quantized_block = np.array([
        [-26, -3, -6, 2, 2, -1, 0, 0],
        [0, -2, -4, 1, 1, 0, 0, 0],
        [-3, 1, 5, -1, -1, 0, 0, 0],
        [-3, 1, 2, -1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]
    )

    expected = np.array(
        [-26,
        -3, 0,
        -3, -2, -6,
        2, -4, 1, -3,
        1, 1, 5, 1, 2,
        -1, 1, -1, 2, 0, 0,
        0, 0, 0, -1, -1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,
        0, 0, 0, 0,
        0, 0, 0,
        0, 0,
        0]
    )
    result = zigzag(quantized_block)
    assert np.allclose(result, np.trim_zeros(expected, trim='b')) == True
    print("[test_zigzag]: OK")
    
test_zigzag()

[test_zigzag]: OK


### b) Huffman

We know need to encode coefficients in the zigzag ordering using JPEG predefined Huffman table for DC/AC coefficients. We used tables from "Digital Image Processing" by Rafael C. Gonzalez & Richard E. Woods page 934.

Let's denote:

- **A**: JPEG coefficients coding categories table
- **B**: JPEG DC coefficients table
- **C**: JPEG AC coefficients table

**<ins>To encode DC coefficient<ins>:** $DC = -26$
- Check in **A** in which category <a style="color:green">CAT</a> $-26$ belongs to: (category 5).
    - This means that -26 needs 5 bits to be represented in binary.
    - Retrieve <a style="color:red">binary representation</a> of number with <a style="color:green">CAT</a> in **A** (<a style="color:red">00101</a>)
    ![](https://cdn.discordapp.com/attachments/844292513665515531/859111745871413278/unknown.png)
- Retrieve codeword in **B** using <a style="color:green">CAT</a> $\to$ codeword = <mark> <a style="color:green">110</a></mark>
    - Conclusion: $DC =$ <mark><a style="color:green">110</a></mark> <a style="color:red">00101</a>
    
**<ins>To encode AC coefficients<ins>:** $AC = -3$ (The second $-3$ on above zigzag ordering)
- Check in **A** in which category <a style="color:green">CAT</a> $-3$ belongs to: (category 2)
  - This means that -3 needs 2 bits to be represented in binary.
    - Retrieve <a style="color:red">binary representation</a> of number with <a style="color:green">CAT</a> in **B** (<a style="color:red">00</a>)
    ![](https://cdn.discordapp.com/attachments/844292513665515531/859113073365614602/unknown.png)
- Compute <a style="color:purple">RUN</a> which correspond to number of 0s precedding AC coeff (<a style="color:purple">RUN=1</a>)
- Retrieve codeword in **C** using <mark><a style="color:purple">RUN</a>/<a style="color:green">CAT</a> = <a style="color:purple">1</a>/<a style="color:green">2</a></mark> $\to$ codeword = <mark>111001</mark>
- Conclusion: $AC = $ <mark>111001</mark><a style="color:red">00</a>

In [22]:
def test_huffman():
    largest_range = list(itertools.product(['0', '1'], repeat=15))

    # Regular Huffman.
    zigzag_order = np.array([-26, -3, 0, -3, -2, -6, 2, -4, 1, -3, 1, 1, 5, 1, 2, -1, 1, -1, 2, 0, 0, 0, 0, 0, -1, -1, 0])
    expected = np.array(["11000101","0100","11100100","0101","100001","0110","100011","001","0100","001","001","100101","001","0110","000","001","000","0110","11110100","000","1010"])

    result = huffman(zigzag_order, largest_range)
    assert len(result) == len(expected)
        
    for res, exp in zip(result, expected):
        assert res == exp
    
    # Huffman with 27 zeros before AC coeff.
    zigzag_order = np.array([-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
    expected = np.array(["11000101", "1111111101110", "11111110101", "1010"])

    result = huffman(zigzag_order, largest_range)
    assert len(result) == len(expected)
    
    for res, exp in zip(result, expected):
        assert res == exp
    
    print("[test_huffman]: OK")

test_huffman()

[test_huffman]: OK


---

# <ins> Decompresion </ins>

JPEG decompression process is the exact inverse of the compression scheme.

![](https://cdn.discordapp.com/attachments/844292513665515531/859105375265226782/unknown.png)

## 1) Huffman$^{-1}$ + Zigzag$^{-1}$

### A) Huffman$^{-1}$

From the bitstream, we have to recover the 8x8 macro-block in zigzag order. To do so, we use a "sliding window" approach where we check if the codeword formed by the sliding window belongs to the table or not. A full 8x8 macro-block is found when reaching the EOB = "1010" (End Of Block) symbol.

![](https://cdn.discordapp.com/attachments/844292513665515531/859134555893399614/unknown.png)

In [24]:
def test_huffman_inv():
    bitstream = "1100010101001110010001011000010110100011001010000100110010100101100000010000110111101000001010"
    largest_range = list(itertools.product(['0', '1'], repeat=15))
    expected = [-26, -3, 0, -3, -2, -6, 2, -4, 1, -3, 1, 1, 5, 1, 2, -1, 1, -1, 2, 0, 0, 0, 0, 0, -1, -1]

    result = next(iter(huffman_inv(bitstream, largest_range)))
    assert len(result) == len(expected)
    assert np.allclose(result, expected) == True
    
    print("[test_huffman_inv]: OK")

test_huffman_inv()

[test_huffman_inv]: OK


### b) Zigzag$^{-1}$

We recreate the 8x8 matrix by re-placing values of the zigzag order.

![](https://cdn.discordapp.com/attachments/844292513665515531/859135707671035944/unknown.png)

In [28]:
def test_zigzag_inv():
    #  Top left corner matrix.
    zigzag_order = [-26, -3, 0, -3, -2, -6, 2, -4, 1, -3, 1, 1, 5, 1, 2, -1, 1, -1, 2, 0, 0, 0, 0, 0, -1, -1]
    expected = np.array([
        [-26, -3, -6,  2,  2, -1,  0,  0],
        [  0, -2, -4,  1,  1,  0,  0,  0],
        [ -3,  1,  5, -1, -1,  0,  0,  0],
        [ -3,  1,  2, -1,  0,  0,  0,  0],
        [  1,  0,  0,  0,  0,  0,  0,  0],
        [  0,  0,  0,  0,  0,  0,  0,  0],
        [  0,  0,  0,  0,  0,  0,  0,  0],
        [  0,  0,  0,  0,  0,  0,  0,  0]]
    )

    result = zigzag_inv(zigzag_order)
    assert np.allclose(result, expected) == True
    
    # Whole matrix.
    zigzag_order = [-42, -1, -7, -6, 0, -38, 3, 4, -1, -3, -9, -2, 7, 0, 7, -4, 2, 0, 2, 1, 7, -3, 1, 5, 2, -2, 0, 0, 4, -1, 2, -1, -1, -2, 0, 1, 0, 1, -1, -1, -1, 1, 1, -3, 0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, -1, 0, -1, 0, -1]
    expected = np.array([
        [-42, -1, -38,  3,  7,  -4,  0,  4],
        [ -7,  0,   4,  0,  2,   0, -1,  1],
        [ -6, -1,   7,  0,  -2,  2,  1, -3],
        [ -3, -2,   2,  2,  -1, -1,  0,  0],
        [ -9,  1,   5, -1,  -1,  0,  0,  0],
        [  7,  1,  -2, -1,  -1,  1,  1,  0],
        [ -3,  0,   1,  0,   1,  0, -1, -1],
        [  1,  0,  -1,  0,   1,  1,  0, -1]]
    )
        
    result = zigzag_inv(zigzag_order)
    assert np.allclose(result, expected) == True

    print("[test_zigzag_inv]: OK")

test_zigzag_inv()

[test_zigzag_inv]: OK


## 2) Reverse quantization

An element-wise multiplication (Hadamard product) is performed to retrieve the matrix before quantization. Note that the retrieved matrix is different from the one in compression since we performed a rounding operation.

![](https://cdn.discordapp.com/attachments/844292513665515531/859135529287286834/unknown.png)

In [30]:
def test_quantization_inv():
    q_block = np.array([
        [-26, -3, -6, 2, 2, -1, 0, 0],
        [0, -2, -4, 1, 1, 0, 0, 0],
        [-3, 1, 5, -1, -1, 0, 0, 0],
        [-3, 1, 2, -1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]
    )

    expected = np.array([
        [-416,  -33,  -60,   32,   48,  -40,    0,    0],
        [   0,  -24,  -56,   19,   26,    0,    0,    0],
        [ -42,   13,   80,  -24,  -40,    0,    0,    0],
        [ -42,   17,   44,  -29,    0,    0,    0,    0],
        [  18,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]
    ])


    result = quantization_inv(q_block, Q_MAT)
    assert np.allclose(result, expected) == True
    print("[test_quantization_inv]: OK")

test_quantization_inv()

[test_quantization_inv]: OK


## 3) DCT$^{-1}$

The reverse DCT is performed to retrieve the initial 8x8 macro-block. We used the following formula:

$$block = DCT^{-1}(\tilde{J}_{Bi^*}) + 128$$

In [33]:
def test_dct_inv():
    dct_block = np.array([
        [-416,  -33,  -60,   32,   48,  -40,    0,    0],
        [   0,  -24,  -56,   19,   26,    0,    0,    0],
        [ -42,   13,   80,  -24,  -40,    0,    0,    0],
        [ -42,   17,   44,  -29,    0,    0,    0,    0],
        [  18,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0]
    ])

    expected = np.array([
        [62, 65, 57, 60, 72, 63, 60, 82],
        [57, 55, 56, 82, 108, 87, 62, 71],
        [58, 50, 60, 111, 148, 114, 67, 65],
        [65, 55, 66, 120, 155, 114, 68, 70],
        [70, 63, 67, 101, 122, 88, 60, 78],
        [71, 71, 64, 70, 80, 62, 56, 81],
        [75, 82, 67, 54, 63, 65, 66, 83],
        [81, 94, 75, 54, 68, 81, 81, 87]
    ])
        
    result = dct_inv(dct_block)
    assert np.allclose(result, expected) == True
    print("[test_dct_inv]: OK")

test_dct_inv()

[test_dct_inv]: OK


## 4) Block combination

We are only left with combining all retrieved 8x8 macro-block to form the original image in compression.
![](https://cdn.discordapp.com/attachments/844292513665515531/859136855672750130/unknown.png)